## EDA LAB

The General Social Survey (GSS) is a bi-annual nationally representative survey of Americans, with almost 7000 different questions asked since the survey began in the 1970s. It has straightforward questions about respondents' demographic information, but also questions like "Does your job regularly require you to perform repetitive or forceful hand movements or involve awkward postures?" or "How often do the demands of your job interfere with your family life?" There are a variety of controversial questions. No matter what you're curious about, there's something interesting in here to check out. The codebook is 904 pages (use CTRL+F to search it).

The data and codebook are available at:
https://gss.norc.org/us/en/gss/get-the-data.html

The datasets are so large that it might make sense to pick the variables you want, and then download just those variables from:
https://gssdataexplorer.norc.org/variables/vfilter

Here is your task:
1. Download a small (5-15) set of variables of interest. (You can also check out `get_gss.ipynb` for some processed data.)
2. Write a short description of the data you chose, and why. (~500 words)
3. Load the data using Pandas. Clean them up for EDA. Do this in this notebook with comments or markdown chunks explaining your choices.
4. Produce some numeric summaries and visualizations.
5. Describe your findings. (500 - 1000 words, or more)

For example, you might want to look at how aspects of a person's childhood family are correlated or not with their career or family choices as an adult. Or how political or religious affiliations correlate with drug use or sexual practices. It's an extremely wide-ranging survey.


Group P: Bacheler Burt, Lily Rademacher, Mia Cachion

We acknowledge that we have used AI to the extent permitted by the UVA AI acceptable use policy. This has been mutually agreed upon. We acknowlegde that we are aware of the possibility of AI producing errors and offensive content and certify that we have taken the necessary measures to ensure that the work produced is both accurate and ethical to the best of our ability.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

Our lab is centered around media consumption and political views. There is significant interplay between the two spheres, stemming from the fact that different media forms and companies have specific slants with respect to political ideology. Media consumption has the power to influence and intensify political views. This dynamic is exemplified by political talk show hosts like Bill Maher and Tucker Carlson, who aim to garner viewership and convince viewers of their political position. Increasingly, politics are grossly intertwined with media presence—political media personalities operate as pseuod-influencers and pseudo-politicians. We hope to analyze political views as a function of media engagement across a few different sectors of the media, as well as incorporate demographic controls such as sex, age, level of education, and region.

The full list of our variable set is as follows:

TVHOURS: Hours per day spent watching television
NEWS: Frequency of reading the newspaper
NEWSFROM: Main source of information about events in the news
PARTYID: Political party identification
POLVIEWS: Political ideology (liberal/conservative scale)
WHOVOTE24: Voting intention in 2024 election (Pre-July 2024)
AGE: Age of respondent
SEX: Sex of respondent
EDUC: Highest year of school completed
REGION: Region of interview
RACE: Race of respondent (White/Black/Other; available for all years, simple to use)
INCOME: Family income (categorical)
REALINC: Family income (continuous in dollars)
In selecting our data, we aimed to capture the multifaceted relationship between media consumption and political attitudes in the United States. The contemporary media landscape is marked by a proliferation of both traditional and digital platforms, each with distinct influences on how individuals encounter and process political information. It's our anecdotal understanding that people often gravitate toward media sources that reinforce their existing beliefs, which can contribute to political polarization. People seek out echo chambers that validate their pre-existing opinions. Digital media, especially social platforms, has the power to expose viewers to a wider array of viewpoints, but it also has the power to isolate people from opposing perspectives by only pushing content to them that they already support.

Our variable set was chosen to reflect these dynamics. By including measures of television, newspaper, and internet use, we can compare the effects of different media types on political attitudes and behaviors. Political variables such as party identification, ideology, and voting behavior allow us to examine not just opinions, but also engagement. Demographic controls (age, sex, education, region, race, and income) are essential for understanding how media and politics intersect across diverse social groups.

We hope to explore questions like: Do certain media habits predict stronger partisan behavior? Are there demographic divides in media use that map onto political ones? How does trust in government relate to both media consumption and socioeconomic status?

The extracted data is stored under "gss_media_pol.csv" in this Github repo.

In [3]:
#2
media = pd.read_csv("gss_media_pol.csv")
print(media.head(10))

   year  id_ age                educ     sex   race             income  \
0  1972    1  23  4 years of college  FEMALE  White  .i:  Inapplicable   
1  1972    2  70          10th grade    MALE  White  .i:  Inapplicable   
2  1972    3  48          12th grade  FEMALE  White  .i:  Inapplicable   
3  1972    4  27  5 years of college  FEMALE  White  .i:  Inapplicable   
4  1972    5  61          12th grade  FEMALE  White  .i:  Inapplicable   
5  1972    6  26  2 years of college    MALE  White  .i:  Inapplicable   
6  1972    7  28   1 year of college    MALE  White  .i:  Inapplicable   
7  1972    8  27  4 years of college    MALE  White  .i:  Inapplicable   
8  1972    9  21          12th grade  FEMALE  Black  .i:  Inapplicable   
9  1972   10  30          12th grade  FEMALE  Black  .i:  Inapplicable   

               region                             partyid           polviews  \
0  East North Central      Independent, close to democrat  .i:  Inapplicable   
1  East North Central    

In [ ]:
#3

In [ ]:
#4

In [ ]:
#5